In [43]:
# Import Packages
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime

In [44]:
# Read in shark data
eat = pd.read_excel("SharkData.xlsx", sheet_name = 0)
targets = pd.read_excel("SharkData.xlsx", sheet_name = 1)
drops = pd.read_excel("SharkData.xlsx", sheet_name = 2)
other_factors = pd.read_excel("SharkData.xlsx", sheet_name = 3)

# Data Cleaning

In [45]:
# Clean eating dataset
eat = eat.iloc[1:,:-1]
eat.rename(columns = {"Unnamed: 14":'Etc. Comments', "Pieces Eaten": "Date", 'Total: ' : 'Total'}, inplace = True)

#get rid of null values at end of dataset
eat = eat.head(582)

# Change date to just date format, not datetime
eat['Date'] = pd.to_datetime(eat['Date']).dt.date



eat.reset_index(drop=True,inplace = True)
eat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total,Etc. Comments
0,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,NaN
1,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,NaN
3,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN
4,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,NaN


In [46]:
# Clean Drops Dataset
drops = drops.iloc[1:,:-1]
drops['Drops'] = pd.to_datetime(drops['Drops']).dt.date
drops.rename(columns = {"Drops" : "Date", "Total: ":"Total"}, inplace = True)
drops.reset_index(drop=True,inplace = True)
drops.tail()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total
577,2020-09-22,0.0,0.0,1.0,4.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,10.0
578,2020-09-24,0.0,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,7.0
579,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0
580,2020-09-27,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,3.0
581,2020-09-29,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,4.0


In [47]:
# Clean Targets Dataset
targets = targets.iloc[1:,:-1]
targets.rename(columns = {"Unnamed: 14":'Etc. Comments', "Targets": "Date", "Total: ": "Total"}, inplace = True)

#get rid of null values at end of dataset
targets = targets.head(582)

# Change date to just date format, not datetime
targets['Date'] = pd.to_datetime(targets['Date']).dt.date

targets.reset_index(drop=True,inplace = True)
targets.tail()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total
577,2020-09-22,6.0,1.0,4.0,6.0,5.0,4.0,7.0,2.0,1.0,5.0,4.0,1.0,46.0
578,2020-09-24,8.0,18.0,2.0,9.0,10.0,10.0,11.0,5.0,6.0,3.0,7.0,9.0,98.0
579,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,3.0,5.0,8.0,24.0
580,2020-09-27,10.0,19.0,9.0,10.0,8.0,16.0,3.0,NaN,NaN,NaN,NaN,NaN,75.0
581,2020-09-29,15.0,5.0,6.0,6.0,4.0,4.0,7.0,4.0,1.0,4.0,4.0,1.0,61.0


# Other Factors Data Cleaning and Dummy Variable Creation for Foods


In [48]:
dummy_factors = other_factors[['Saury', 'Blue Runner',
                'Squid', 'Mackerel', 'Herring', 'Sardine',
                'Mazuri Vitamins', 'Garlic', 'Salmon', 'Bonito', 'Bluefish', 'Mahi',
                    'Goggle Eye', 'Humbolt Squid']]
other_dummies = other_factors[['BT/SB Location (1-6)','GR Location (1-6)']]
other_dummies.rename(columns= {"BT/SB Location (1-6)": "BT_SB_Location", "GR Location (1-6)": "GR_Location"}, inplace = True)
for i in dummy_factors:
    dummy_factors[i] = dummy_factors[i].fillna(0)
dummy_factors.rename(columns = {"Blue Runner": "Blue_Runner", "Mazuri Vitamins": "Mazuri_Vitamins", 
                               "Goggle Eye": "Goggle_Eye", "Humbolt Squid": "Humbolt_Squid"}, inplace = True)

/home/stats/.local/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Combine Datasets to get all factors to perform regression on

In [49]:
# Add seasons variables
def season_of_date(date):
    date = pd.to_datetime(date)
    year = str(date.year)
    seasons = {'spring': pd.date_range(start=year+'/03/21', end=year+'/06/20'),
               'summer': pd.date_range(start=year+'/06/21', end=year+'/09/22'),
               'fall': pd.date_range(start=year+'/09/23', end=year+'/12/20')}
    if date in seasons['spring']:
        return 'spring'
    if date in seasons['summer']:
        return 'summer'
    if date in seasons['fall']:
        return 'fall'
    else:
        return 'winter'

In [50]:
eat_summer = eat[eat.Date.map(season_of_date)=="summer"]
eat_summer['summer'] = 1
eat_summer['fall'] = 0
eat_summer['spring'] = 0
eat_fall = eat[eat.Date.map(season_of_date)=="fall"]
eat_fall['summer'] = 0
eat_fall['fall'] = 1
eat_fall['spring'] = 0
eat_winter = eat[eat.Date.map(season_of_date)=="winter"]
eat_winter['summer'] = 0
eat_winter['fall'] = 0
eat_winter['spring'] = 0
eat_spring = eat[eat.Date.map(season_of_date)=="spring"]
eat_spring['summer'] = 0
eat_spring['fall'] = 0
eat_spring['spring'] = 1


/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [51]:
eat_all_seasons = pd.concat([eat_summer,eat_fall,eat_winter,eat_spring])
eat_all_seasons_sort = eat_all_seasons.sort_values(by=["Date"])
eat_season_vars = eat_all_seasons_sort[['summer', 'fall', 'spring']]

In [52]:
drops_summer = drops[eat.Date.map(season_of_date)=="summer"]
drops_summer['summer'] = 1
drops_summer['fall'] = 0
drops_summer['spring'] = 0
drops_fall = drops[eat.Date.map(season_of_date)=="fall"]
drops_fall['summer'] = 0
drops_fall['fall'] = 1
drops_fall['spring'] = 0
drops_winter = drops[eat.Date.map(season_of_date)=="winter"]
drops_winter['summer'] = 0
drops_winter['fall'] = 0
drops_winter['spring'] = 0
drops_spring = drops[eat.Date.map(season_of_date)=="spring"]
drops_spring['summer'] = 0
drops_spring['fall'] = 0
drops_spring['spring'] = 1

/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [53]:
drops_all_seasons = pd.concat([drops_summer,drops_fall,drops_winter,drops_spring])
drops_all_seasons_sort = drops_all_seasons.sort_values(by=["Date"])
drops_season_vars = drops_all_seasons_sort[['summer', 'fall', 'spring']]

In [54]:
targets_summer = targets[eat.Date.map(season_of_date)=="summer"]
targets_summer['summer'] = 1
targets_summer['fall'] = 0
targets_summer['spring'] = 0
targets_fall = targets[eat.Date.map(season_of_date)=="fall"]
targets_fall['summer'] = 0
targets_fall['fall'] = 1
targets_fall['spring'] = 0
targets_winter = targets[eat.Date.map(season_of_date)=="winter"]
targets_winter['summer'] = 0
targets_winter['fall'] = 0
targets_winter['spring'] = 0
targets_spring = targets[eat.Date.map(season_of_date)=="spring"]
targets_spring['summer'] = 0
targets_spring['fall'] = 0
targets_spring['spring'] = 1

/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/stats/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [55]:
targets_all_seasons = pd.concat([targets_summer,targets_fall,targets_winter,targets_spring])
targets_all_seasons_sort = targets_all_seasons.sort_values(by=["Date"])
targets_season_vars = targets_all_seasons_sort[['summer', 'fall', 'spring']]

In [56]:
# combine eat data
eat_concat1 = pd.concat([eat,dummy_factors,other_dummies, eat_season_vars], axis = 1)
eat_concat = pd.concat([eat_concat1, other_factors["Temperature"]], axis = 1)
eat_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,BT_SB_Location,GR_Location,summer,fall,spring,Temperature
0,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
1,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,...,0.0,0.0,0.0,0.0,NaN,3.0,0,1,0,73.6
3,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,NaN,0,1,0,73.6
4,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.4


In [57]:
# Add eat variable for Covid shut down
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
conditions = [ (eat_concat['Date'] >= start_date) & (eat_concat['Date'] <= end_date),
              (eat_concat['Date'] < start_date) | (eat_concat['Date'] > end_date)]
values = [1, 0]
eat_concat["covid"] = np.select(conditions, values)

In [58]:
# Add eat variable for light training
light_start = pd.to_datetime('02-15-2018')
conditions_light = [ (eat_concat['Date'] >= light_start), (eat_concat['Date'] < light_start)]
values_light = [1, 0]
eat_concat["light_training"] = np.select(conditions_light, values_light)

In [59]:
# combine drops data
drops_concat1 = pd.concat([drops,dummy_factors,other_dummies, drops_season_vars], axis = 1)
drops_concat = pd.concat([drops_concat1, other_factors["Temperature"]], axis = 1)
drops_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,BT_SB_Location,GR_Location,summer,fall,spring,Temperature
0,2017-12-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
1,2017-12-07,0.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,0.0,0.0,0.0,0.0,NaN,3.0,0,1,0,73.6
3,2017-12-10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,NaN,0,1,0,73.6
4,2017-12-12,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.4


In [60]:
# Add drop variable for Covid shut down
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
conditions = [ (drops_concat['Date'] >= start_date) & (drops_concat['Date'] <= end_date),
              (drops_concat['Date'] < start_date) | (drops_concat['Date'] > end_date)]
values = [1, 0]
drops_concat["covid"] = np.select(conditions, values)

In [61]:
# Add drop variable for light training
light_start = pd.to_datetime('02-15-2018')
conditions_light = [ (drops_concat['Date'] >= light_start), (drops_concat['Date'] < light_start)]
values_light = [1, 0]
drops_concat["light_training"] = np.select(conditions_light, values_light)

In [62]:
# combine Target data
targets_concat1 = pd.concat([targets,dummy_factors,other_dummies, targets_season_vars], axis = 1)
targets_concat = pd.concat([targets_concat1, other_factors["Temperature"]], axis = 1)
targets_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,BT_SB_Location,GR_Location,summer,fall,spring,Temperature
0,2017-12-05,3.0,0.0,4.0,3.0,0.0,2.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
1,2017-12-07,4.0,5.0,2.0,2.0,5.0,3.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.2
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,...,0.0,0.0,0.0,0.0,NaN,3.0,0,1,0,73.6
3,2017-12-10,4.0,7.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,NaN,0,1,0,73.6
4,2017-12-12,3.0,1.0,3.0,2.0,4.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,1.0,3.0,0,1,0,73.4


In [63]:
# Add target variable for Covid shut down
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
conditions = [ (targets_concat['Date'] >= start_date) & (targets_concat['Date'] <= end_date),
              (targets_concat['Date'] < start_date) | (targets_concat['Date'] > end_date)]
values = [1, 0]
targets_concat["covid"] = np.select(conditions, values)

In [64]:
# Add target variable for light training
light_start = pd.to_datetime('02-15-2018')
conditions_light = [ (targets_concat['Date'] >= light_start), (targets_concat['Date'] < light_start)]
values_light = [1, 0]
targets_concat["light_training"] = np.select(conditions_light, values_light)

# Subset Creation

In [65]:

# All SS Subset
All_SS = eat_concat[['Ross', 'Chandler']]

# All BT Subset
All_BT = eat_concat[['BT1', 'BT2', 'BT3', 'BT4', 'BT5']]

# All GR Subset
All_GR = eat_concat[['GR1', 'GR2', 'GR3', 'GR4', 'GR5']]

# Male Subset
Male = eat_concat[["BT1","BT5","GR1","Ross","Chandler"]]

# Female Subset
female = eat_concat[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]]

# Append all subsets to original datasets
# eat dataset
eat_concat["All_GR"] = eat['GR1'] + eat['GR2'] + eat['GR3'] + eat['GR4'] + eat['GR5']
eat_concat["All_BT"] = eat['BT1'] + eat['BT2'] + eat['BT3'] + eat['BT4'] + eat['BT5']
eat_concat["All_SS"] = eat['Ross'] + eat['Chandler']
eat_concat["male"] = eat[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_concat["female"] = eat[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

# drop dataset
drops_concat["All_GR"] = drops['GR1'] + drops['GR2'] + drops['GR3'] + drops['GR4'] + drops['GR5']
drops_concat["All_BT"] = drops['BT1'] + drops['BT2'] + drops['BT3'] + drops['BT4'] + drops['BT5']
drops_concat["All_SS"] = drops['Ross'] + drops['Chandler']
drops_concat["male"] = drops[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
drops_concat["female"] = drops[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

# target dataset
targets_concat["All_GR"] = targets['GR1'] + targets['GR2'] + targets['GR3'] + targets['GR4'] + targets['GR5']
targets_concat["All_BT"] = targets['BT1'] + targets['BT2'] + targets['BT3'] + targets['BT4'] + targets['BT5']
targets_concat["All_SS"] = targets['Ross'] + targets['Chandler']
targets_concat["male"] = targets[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
targets_concat["female"] = targets[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)


# Create Tables for Poisson Regression Time Series For Pieces Eaten For All Sharks

In [66]:
# create additional time variables for eating
eat_concat['Date'] = pd.to_datetime(eat_concat['Date'])
eat_concat['Day_of_week'] = pd.to_datetime(eat_concat['Date']).dt.dayofweek
eat_concat['Month'] = pd.to_datetime(eat_concat['Date']).dt.month
eat_concat['Day'] = pd.to_datetime(eat_concat['Date']).dt.day
eat_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,covid,light_training,All_GR,All_BT,All_SS,male,female,Day_of_week,Month,Day
0,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,...,0,0,6.0,22.0,5.0,12.0,21.0,1,12,5
1,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,2.0,1.0,1.0,2.0,3,12,7
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,...,0,0,6.0,NaN,NaN,2.0,4.0,5,12,9
3,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,...,0,0,NaN,8.0,6.0,7.0,7.0,6,12,10
4,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,...,0,0,7.0,16.0,4.0,9.0,18.0,1,12,12


In [67]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concat)) < 0.8
eat_train = eat_concat[mask]
eat_test = eat_concat[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=464
Testing data set length=118


In [68]:
# setup Poisson regression expression
expr = """Total ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer + fall + spring"""

In [69]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [70]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Total   No. Observations:                  239
Model:                            GLM   Df Residuals:                      213
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -806.91
Date:                Fri, 04 Dec 2020   Deviance:                       407.82
Time:                        20:55:07   Pearson chi2:                     379.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.8140      1.243     

Day of week, Month, temperature, Saury, Blue_runner, Herring, light training, summer, and fall were all significant predictors for the number of pieces eaten for all sharks

# Poisson Regression for Number of Drops for All Sharks

In [71]:
# create additional time variables for dropping
drops_concat['Date'] = pd.to_datetime(drops_concat['Date'])
drops_concat['Day_of_week'] = pd.to_datetime(drops_concat['Date']).dt.dayofweek
drops_concat['Month'] = pd.to_datetime(drops_concat['Date']).dt.month
drops_concat['Day'] = pd.to_datetime(drops_concat['Date']).dt.day
drops_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,covid,light_training,All_GR,All_BT,All_SS,male,female,Day_of_week,Month,Day
0,2017-12-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,2.0,1.0,0.0,0.0,3.0,1,12,5
1,2017-12-07,0.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,...,0,0,2.0,7.0,1.0,4.0,6.0,3,12,7
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,0,0,1.0,NaN,NaN,0.0,1.0,5,12,9
3,2017-12-10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,...,0,0,NaN,1.0,0.0,1.0,0.0,6,12,10
4,2017-12-12,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0,0,1.0,2.0,0.0,0.0,3.0,1,12,12


In [72]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concat)) < 0.8
drops_train = drops_concat[mask]
drops_test = drops_concat[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=470
Testing data set length=112


In [73]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [74]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Total   No. Observations:                  234
Model:                            GLM   Df Residuals:                      208
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -554.58
Date:                Fri, 04 Dec 2020   Deviance:                       370.02
Time:                        20:55:08   Pearson chi2:                     385.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.5056      2.707     

Day of week, Month, Temperature, Blue runner, Mackerel, Garlic, and Bluefish were all significant predictors for the number of drops for all sharks

# Poisson Regression for Number of Targets for All Sharks

In [75]:
# create additional time variables for Targeting
targets_concat['Date'] = pd.to_datetime(targets_concat['Date'])
targets_concat['Day_of_week'] = pd.to_datetime(targets_concat['Date']).dt.dayofweek
targets_concat['Month'] = pd.to_datetime(targets_concat['Date']).dt.month
targets_concat['Day'] = pd.to_datetime(targets_concat['Date']).dt.day
targets_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,covid,light_training,All_GR,All_BT,All_SS,male,female,Day_of_week,Month,Day
0,2017-12-05,3.0,0.0,4.0,3.0,0.0,2.0,2.0,1.0,1.0,...,0,0,8.0,11.0,3.0,10.0,12.0,1,12,5
1,2017-12-07,4.0,5.0,2.0,2.0,5.0,3.0,5.0,0.0,0.0,...,0,0,4.0,17.0,9.0,16.0,14.0,3,12,7
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,...,0,0,8.0,NaN,NaN,1.0,7.0,5,12,9
3,2017-12-10,4.0,7.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,...,0,0,NaN,4.0,11.0,11.0,4.0,6,12,10
4,2017-12-12,3.0,1.0,3.0,2.0,4.0,1.0,1.0,1.0,1.0,...,0,0,11.0,11.0,4.0,9.0,17.0,1,12,12


In [76]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concat)) < 0.8
targets_train = targets_concat[mask]
targets_test = targets_concat[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=460
Testing data set length=122


In [77]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [78]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Total   No. Observations:                  233
Model:                            GLM   Df Residuals:                      207
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1372.9
Date:                Fri, 04 Dec 2020   Deviance:                       1349.2
Time:                        20:55:09   Pearson chi2:                 1.37e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           2.7058      0.726     

Day of week, Day of month, Blue runner, Mackerel, Herring, Mazuri vitamins, garlic, salmon, humbolt squid, covid, spring, light training were all significant predictors for number of targets for all sharks

# Poisson Regression Across Different Species of Sharks

# Sandbar Sharks

In [79]:
expr = """All_SS ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer + fall + spring"""

In [80]:
eat_concatSS = eat_concat[eat_concat['All_SS'].notna()]

In [81]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatSS)) < 0.8
eat_train = eat_concatSS[mask]
eat_test = eat_concatSS[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=355
Testing data set length=84


Eating:

In [82]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [83]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_SS   No. Observations:                  232
Model:                            GLM   Df Residuals:                      206
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -587.14
Date:                Fri, 04 Dec 2020   Deviance:                       411.17
Time:                        20:55:09   Pearson chi2:                     373.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           8.0682      2.565     

Garlic, Bonito, Bluefish, Location,summer, spring and light training were all significant predictors for the number of fish eaten for Sandbars

Dropping:

In [84]:
drops_concatSS = drops_concat[drops_concat['All_SS'].notna()]

In [85]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatSS)) < 0.8
drops_train = drops_concatSS[mask]
drops_test = drops_concatSS[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=363
Testing data set length=75


In [86]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [87]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_SS   No. Observations:                  242
Model:                            GLM   Df Residuals:                      216
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -173.88
Date:                Fri, 04 Dec 2020   Deviance:                       213.96
Time:                        20:55:10   Pearson chi2:                     264.
No. Iterations:                    21                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.4741     10.251     

Goggle eye, spring, and light training were the only significant predictors for the number of drops for the Sandbar sharks

Targets:

In [88]:
targets_concatSS = targets_concat[targets_concat['All_SS'].notna()]

In [89]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatSS)) < 0.8
targets_train = targets_concatSS[mask]
targets_test = targets_concatSS[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=355
Testing data set length=82


In [90]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [91]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_SS   No. Observations:                  235
Model:                            GLM   Df Residuals:                      209
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -954.18
Date:                Fri, 04 Dec 2020   Deviance:                       863.05
Time:                        20:55:10   Pearson chi2:                     844.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.8127      1.572     

Mazuri Vitamins, garlic, salmon, bluefish, goggle eye, location, and light training were all significant predictors for number of targets for sandbar sharks

# Blacktip Sharks

Eating:

In [92]:
expr = """All_BT ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer + fall + spring"""

In [93]:
eat_concatBT = eat_concat[eat_concat['All_BT'].notna()]

In [94]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatBT)) < 0.8
eat_train = eat_concatBT[mask]
eat_test = eat_concatBT[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=351
Testing data set length=85


In [95]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [96]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_BT   No. Observations:                  234
Model:                            GLM   Df Residuals:                      208
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -702.90
Date:                Fri, 04 Dec 2020   Deviance:                       377.06
Time:                        20:55:11   Pearson chi2:                     372.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -3.0298      1.646     

Temperature, Blue Runner, Mackerel, Mazuri vitamins, and light training were all significant predictors for pieces eaten for Black Tips

Dropping:

In [97]:
drops_concatBT = drops_concat[drops_concat['All_BT'].notna()]

In [98]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatBT)) < 0.8
drops_train = drops_concatBT[mask]
drops_test = drops_concatBT[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=350
Testing data set length=86


In [99]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [100]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_BT   No. Observations:                  236
Model:                            GLM   Df Residuals:                      210
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -489.57
Date:                Fri, 04 Dec 2020   Deviance:                       330.41
Time:                        20:55:12   Pearson chi2:                     326.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.5274      3.543     

Day of week, Temperature, Saury, Garlic, Bluefish, humbolt squid, covid, and light training were all significant predictors for the number of drops for Black Tips.

Targeting:

In [101]:
targets_concatBT = targets_concat[targets_concat['All_BT'].notna()]

In [102]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatBT)) < 0.8
targets_train = targets_concatBT[mask]
targets_test = targets_concatBT[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=332
Testing data set length=105


In [103]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [104]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_BT   No. Observations:                  216
Model:                            GLM   Df Residuals:                      190
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1039.4
Date:                Fri, 04 Dec 2020   Deviance:                       922.24
Time:                        20:55:12   Pearson chi2:                     924.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.1307      1.079     

Month, Day of month, Saury, Squid, Mackerel, Herring, Salmon, Bonito, location, covid, summer, spring and light training were all significant predictors for the number of targets for Black Tips

# Gray Reef Sharks

Eating:

In [105]:
expr = """All_GR ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer + fall + spring"""

In [106]:
eat_concatGR = eat_concat[eat_concat['All_GR'].notna()]

In [107]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatGR)) < 0.8
eat_train = eat_concatGR[mask]
eat_test = eat_concatGR[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=338
Testing data set length=99


In [108]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [109]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_GR   No. Observations:                  221
Model:                            GLM   Df Residuals:                      195
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -595.26
Date:                Fri, 04 Dec 2020   Deviance:                       403.77
Time:                        20:55:13   Pearson chi2:                     365.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.7317      2.385     

Saury, Blue runner, Squid, Mackerel, Herring, Bonito, Humbolt Squid, summer, fall, and light training were all significant predictors for number of fish eaten for Gray Reefs

Dropping:

In [110]:
drops_concatGR = drops_concat[drops_concat['All_GR'].notna()]

In [111]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatGR)) < 0.8
drops_train = drops_concatGR[mask]
drops_test = drops_concatGR[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=358
Testing data set length=80


In [112]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [113]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_GR   No. Observations:                  237
Model:                            GLM   Df Residuals:                      211
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -301.39
Date:                Fri, 04 Dec 2020   Deviance:                       307.31
Time:                        20:55:13   Pearson chi2:                     315.
No. Iterations:                    19                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           7.8540      6.374     

Blue runner and light training were the only significant predictors for number of drops for Gray Reefs

Targeting:

In [114]:
targets_concatGR = targets_concat[targets_concat['All_GR'].notna()]

In [115]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatGR)) < 0.8
targets_train = targets_concatGR[mask]
targets_test = targets_concatGR[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=357
Testing data set length=80


In [116]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [117]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 All_GR   No. Observations:                  239
Model:                            GLM   Df Residuals:                      213
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -945.30
Date:                Fri, 04 Dec 2020   Deviance:                       836.99
Time:                        20:55:14   Pearson chi2:                     808.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -4.3806      1.426     

Day of week, Temperature, Saury, Blue runner, Squid, Mackerel, Herring, Mazuri vitamins, Bluefish, humbolt squid, location, covid, fall, and light training were all significant predictors for the number of targets for Gray Reefs

# Poisson Regression for Each Gender of Shark

# Male

Eating:

In [118]:
expr = """male ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer + fall + spring"""

In [119]:
eat_concatM = eat_concat[eat_concat['male'].notna()]

In [120]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatM)) < 0.8
eat_train = eat_concatM[mask]
eat_test = eat_concatM[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=478
Testing data set length=104


In [121]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [122]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   male   No. Observations:                  240
Model:                            GLM   Df Residuals:                      214
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -682.13
Date:                Fri, 04 Dec 2020   Deviance:                       332.05
Time:                        20:55:14   Pearson chi2:                     317.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.3684      1.625     

Month, Herring, Bluefish, BT_SB location, summer, and light training were all significant predictors for the number of fish eaten for male sharks

Dropping:

In [123]:
drops_concatM = drops_concat[drops_concat['male'].notna()]

In [124]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatM)) < 0.8
drops_train = drops_concatM[mask]
drops_test = drops_concatM[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=461
Testing data set length=121


In [125]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [126]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   male   No. Observations:                  234
Model:                            GLM   Df Residuals:                      208
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -389.62
Date:                Fri, 04 Dec 2020   Deviance:                       294.88
Time:                        20:55:15   Pearson chi2:                     272.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.7858      4.381     

Day of week, Squid, BT_SB location, and light training were all significant predictors for number of pieces dropped for male sharks

Targetting:

In [127]:
targets_concatM = targets_concat[targets_concat['male'].notna()]

In [128]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatM)) < 0.8
targets_train = targets_concatM[mask]
targets_test = targets_concatM[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=471
Testing data set length=111


In [129]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [130]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                   male   No. Observations:                  233
Model:                            GLM   Df Residuals:                      207
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -1104.8
Date:                Fri, 04 Dec 2020   Deviance:                       981.53
Time:                        20:55:15   Pearson chi2:                     974.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           4.7812      1.033     

Sardine, Day of week, Month, Day of month, Squid, Mackerel, Herring, Mazuri Vitamins, Salmon, Bonito, Goggle eye, humbolt squid, BT_SB location, covid, spring, and light training were all significant predictors for the number of targets for male sharks

# Female

Eating:

In [131]:
expr = """female ~ Day_of_week + Month + Day + Temperature + Saury + Blue_Runner + 
Squid + Mackerel + Herring + Sardine + Mazuri_Vitamins + Garlic + Salmon + Bonito + Bluefish +
Mahi + Goggle_Eye + Humbolt_Squid + BT_SB_Location + GR_Location + covid + light_training + summer + fall + spring"""

In [132]:
eat_concatF = eat_concat[eat_concat['female'].notna()]

In [133]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concatF)) < 0.8
eat_train = eat_concatF[mask]
eat_test = eat_concatF[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=467
Testing data set length=115


In [134]:
y_train, X_train = dmatrices(expr, eat_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, eat_test, return_type='dataframe')

In [135]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 female   No. Observations:                  227
Model:                            GLM   Df Residuals:                      201
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -679.97
Date:                Fri, 04 Dec 2020   Deviance:                       361.89
Time:                        20:55:16   Pearson chi2:                     345.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.4724      1.541     

Sardine, Temperature, Saury, Blue runner, Mackerel, Herring, humbolt squid, summer, fall, and BT_SB locations were all significant predictors for number of pieces eaten for female sharks

Dropping:

In [136]:
drops_concatF = drops_concat[drops_concat['female'].notna()]

In [137]:
# Create training and testing datasets
mask = np.random.rand(len(drops_concatF)) < 0.8
drops_train = drops_concatF[mask]
drops_test = drops_concatF[~mask]
print('Training data set length='+str(len(drops_train)))
print('Testing data set length='+str(len(drops_test)))

Training data set length=475
Testing data set length=107


In [138]:
y_train, X_train = dmatrices(expr, drops_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, drops_test, return_type='dataframe')

In [139]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 female   No. Observations:                  245
Model:                            GLM   Df Residuals:                      219
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -481.51
Date:                Fri, 04 Dec 2020   Deviance:                       356.12
Time:                        20:55:17   Pearson chi2:                     349.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           9.8934      3.830     

Sardine, Day of week, Month, Temperature, Blue runner, Squid, Herring, Bluefish, BT_SB Location, GR_location, and covid were all significant predictors for the number of pieces dropped for female sharks

Targetting:

In [140]:
targets_concatF = targets_concat[targets_concat['female'].notna()]

In [141]:
# Create training and testing datasets
mask = np.random.rand(len(targets_concatF)) < 0.8
targets_train = targets_concatF[mask]
targets_test = targets_concatF[~mask]
print('Training data set length='+str(len(targets_train)))
print('Testing data set length='+str(len(targets_test)))

Training data set length=451
Testing data set length=131


In [142]:
y_train, X_train = dmatrices(expr, targets_train, return_type='dataframe')
y_test, X_test = dmatrices(expr, targets_test, return_type='dataframe')

In [143]:
# Fit model and print results of model
poisson_training_results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(poisson_training_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                 female   No. Observations:                  225
Model:                            GLM   Df Residuals:                      199
Model Family:                 Poisson   Df Model:                           25
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -914.57
Date:                Fri, 04 Dec 2020   Deviance:                       645.64
Time:                        20:55:17   Pearson chi2:                     645.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.0003      1.071     

Temperature, Blue runner, Mackerel, Herring, Bluefish, Humbolt squid, BT_SB location, spring, and light training were all significant predictors for the number of targets for female sharks